# Название проекта

**Описание проекта:**

**Цель исследования —**

**Описание данных:**

# Содержание проекта:

 - [Шаг 1. Подключение к базе. Загрузка таблицы sql](#getting-data)
 - [Шаг 2. Первичное исследование таблиц](#exploratory-data-analysis)
 - [Шаг 3. Статистический анализ факторов ДТП](#statistical-factor-analysis)
 - [Шаг 4. Создание модели для оценки водительского риска](#model-creation)
 - [Шаг 5. Поиск лучшей модели и анализ важности факторов ДТП](#best-model-search)
 - [Шаг 6. Проверка лучшей модели в работе](#best-model-test)
 - [Шаг 7. Выводы](#general-conclusion)

Импортируем полезные библиотеки, объявим константы и зададим параметры по умолчанию

In [ ]:
!pip install phik optuna torchvision spacy~=3.2.6 scikit-learn~=1.5.0

# data manipulation
import numpy as np
import pandas as pd

# database
import psycopg2

# plotting
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

# math and optimization
import math
import scipy.stats as st
import phik
import optuna
from phik.report import plot_correlation_matrix
from statsmodels.tsa.stattools import adfuller

# utility
import itertools
import copy
import os
from IPython.core.display import display, HTML
from tqdm import tqdm
from functools import partial

# time series
import calendar
import datetime
from time import time
from statsmodels.tsa.seasonal import seasonal_decompose

# sklearn
from sklearn import set_config
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import train_test_split, KFold, GroupShuffleSplit 
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, ParameterGrid
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, SGDRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, log_loss, f1_score, fbeta_score
from sklearn.metrics import roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# lightgbm
from lightgbm import LGBMClassifier, LGBMRegressor, plot_importance

# neural networks
import torch
import torch.nn as nn

# natural language processing
import re
import spacy
import nltk
from nltk.corpus import stopwords

# computer vision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, GlobalAveragePooling2D

# constant values
RANDOM_STATE = 884002
SMALL_SIZE = 12
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

# set default values
display(HTML("<style>.container { width:75% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.rc('font', size=SMALL_SIZE)                                    # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)                              # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)                              # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)                              # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)                              # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)                              # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)                            # fontsize of the figure title
plt.rc('figure', figsize=(18, 12))                                 # controls figure size
sns.set(rc={'figure.figsize':(18, 12)})
tqdm.pandas()
set_config(display='diagram')
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#Функция для исследовательского анализа данных
def exploratory_data_analysis(table, bad_columns):
    good_columns = table.columns.difference(bad_columns)
    good_table = table[good_columns]
    
    # Ознакомимся с набором данных
    display(good_table.head())
    
    # Отобразим информацию для краткого обзора данных
    good_table.info()
    
    # Отобразим таблицу с описательной статистикой столбцов
    display(good_table.describe())
    
    # Отобразим количественные и категориальные переменные соответствующими методами
    fig = plt.figure(figsize=(24, 12))
    plt.subplots_adjust(wspace=0.25, hspace=0.8)

    for i, col in enumerate(good_columns):
        ax = fig.add_subplot(4, 3, i + 1)

        if np.issubdtype(data[col].dtype, np.number):
            good_table[col].plot(kind='hist')
        else:
            good_table[col].value_counts().plot(kind='bar')

        ax.set_title(col)
        ax.grid(visible=True)
        
    # Отобразим таблицу с попарными корреляциями столбцов    
    phik_overview = good_table[good_columns].phik_matrix(interval_cols=[''])
    plot_correlation_matrix(phik_overview.values, x_labels=phik_overview.columns, y_labels=phik_overview.index, 
                            vmin=0, vmax=1, color_map='Blues',
                            title=r'correlation $\phi_K$', fontsize_factor=1.5, figsize=(24, 12))
    plt.tight_layout()

## Загрузка и разведочный анализ данных

### Подключимся к базе данных

In [ ]:
database_config = {
    'user': '',
    'password': '',
    'host': '',
    'port': ,
    'dbname': ''
} 
 
conn = psycopg2.connect(**database_config) 
cur = conn.cursor() 

### Проверим соответствует ли количество таблиц условию задачи:

In [ ]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema=''")
table_names = pd.Series(data=[data_tuple[0] for data_tuple in cur.fetchall()], name='table_names')
table_names

### Проверим все ли таблицы содержат данные:

In [ ]:
table_not_empty = pd.Series(index=table_names.values, dtype='bool', name='table_not_empty')
for table_name in table_names.values:
    cur.execute(f"SELECT EXISTS(SELECT 1 FROM .{table_name})")
    table_not_empty[table_name] = cur.fetchall()[0][0]
    
table_not_empty

### Загрузим все таблицы:

In [ ]:
tables = {}

for table_name in table_names:
    cur.execute(f"SELECT * FROM .{table_name}")
    tables[table_name] = pd.DataFrame(data=cur.fetchall(),
                                      columns=[desc[0] for desc in cur.description])

### Получим данные безопасным способом при помощи конструкции try-except:

In [ ]:
try:
    data = pd.read_csv('/datasets/real_estate_data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv')

### Ознакомимся с набором данных:

In [ ]:
exploratory_data_analysis(data, [])

### Закроем соединение с базой данных:

In [ ]:
cur.close() 
conn.close()

### Краткий вывод:
В данных были обнаружены следующие проблемы (или их отсутствие):
- в данных присутствуют/отсутствуют неинформативные столбцы
- в данных присутствуют/отсутствуют нарушения правила хорошего стиля в названиях столбцов
- в данных присутствуют/отсутствуют несоответствия типов в столбцах ``
- в данных присутствуют/отсутствуют пропуски в столбцах
- в данных присутствуют/отсутствуют явные и неявные дубликаты
- в данных присутствуют/отсутствуют аномальные значения
- в данных присутствуют/отсутствуют намёки на мультиколлинеарность
- в данных присутствуют/отсутствуют намёки на дисбаланс классов в целевом признаке

## Предобработка данных

In [ ]:
# check
data.columns

### Исправим нарушения правил хорошего стиля в названиях столбцов:
* несколько слов в названии запишем в «змеином_регистре»
* все символы сделаем строчными
* устраним пробелы

In [ ]:
data.rename(inplace=True, columns={})

### Исправим несоответствия типов в столбцах:

In [ ]:
data = data.astype({})

# check
data.dtypes

### Добавим новые столбцы:

Категоризуем данные

### Удалим неинформативные столбцы, которые не несут ценности для прогноза::

In [ ]:
data.drop([''], axis=1, inplace=True)

# check
data.columns

### Обработаем пропущенные значения:

Сначала посчитаем, сколько в таблице пропущенных значений

In [ ]:
data.isna().sum()

Обработаем категориальные пропуски

Различают следующие 3 механизма формирования пропусков:
1. **MCAR**. Механизм формирования пропусков, при котором вероятность пропуска для каждой записи набора одинакова. Столбцы, которые имеют такой механизм формирования пропусков:
    - ` `
2. **MAR**. Механизм формирования пропусков, при котором вероятность пропуска может быть определена на основе другой имеющейся в наборе данных информации, не содержащей пропуски. Столбцы, которые имеют такой механизм формирования пропусков:
    - ` `
3. **MNAR**. Механизм формирования пропусков, при котором вероятность пропуска могла бы быть описана на основе других атрибутов, но информация по этим атрибутам в наборе данных отсутствует (например, объект недвижимости может иметь подземный этаж или чердак, но такая информация в наборе данных отсутствует). Столбцы, которые имеют такой механизм формирования пропусков:  
    - ` `

Обработаем количественные пропуски

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения

In [ ]:
data.isna().sum()

### Обработаем дубликаты:

Сначала обработаем неявные дубликаты

Выведем на экран количество полных строк-дубликатов

In [ ]:
data.duplicated().sum()

Удалим полные дубликаты из таблицы

In [ ]:
data = data.drop_duplicates()

Выведем на экран неполные дубликаты по подмножеству стобцов

In [ ]:
data[data.duplicated(subset=[])]

Удалим неполные дубликаты из таблицы

In [ ]:
data = data.drop_duplicates(subset=[])

Ещё раз посчитаем явные дубликаты в таблице и убедимся, что полностью от них избавились

In [ ]:
data.duplicated().sum()

### Обработаем аномальные значения:

Учтем особенности фильтрации `pandas`, чтобы не потерять записи с пропусками

### Краткий вывод:
В данных были устранены следующие проблемы:
- нарушения правил хорошего стиля в названиях столбцов
- несоответствия типов в столбцах
- неинформативные признаки, которые не несут ценности для прогноза
- пропуски в столбцах ``
- явные и неявные дубликаты
- аномальные значения в столбцах ``

## Исследовательский анализ данных:

### Исследуем баланс классов:

### Проанализируем как целевой признак `` связан со всеми остальными признаками:

In [ ]:
corr_data = data.corr()[''].sort_values(key=abs)
abs(corr_data).plot(kind='barh', 
                    fontsize=14,
                    color=(corr_data > 0).map({True: 'g', False: 'r'}),
                    edgecolor='black',
                    title='',
                    grid=True);

### Разделим исходные данные на обучающую, валидационную и тестовую выборки в соотношении $60\%$/$20\%$/$20\%$:

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=RANDOM_STATE, stratify=data.is_ultra)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=RANDOM_STATE, stratify=train_data.is_ultra)

# check
split = pd.DataFrame(index=['train', 'valid', 'test'],
                     columns=['size', 'size_proportion'],
                     data=[[dataset.shape[0], round(dataset.shape[0] / data.shape[0] * 100, 2)], 
                           for dataset in [train_data, valid_data, test_data]])

display(split)

# Разделим обучающую выборку
train_features = train_data.drop([''], axis=1) 
train_target = train_data['']

# Разделим валидационную выборку
valid_features = valid_data.drop([''], axis=1)
valid_target = valid_data['']

# Разделим тестовую выборку
test_features = test_data.drop([''], axis=1)
test_target = test_data['']

### Краткий вывод:

## Обучение моделей:

Создадим объекты, которые понадобятся нам в дальнейшем

In [ ]:
powers_of_two = [2**i for i in range(1, 7)]
cv_results = pd.DataFrame(columns=['MAE'])
model_params = dict()

### Создадим базовый пайплайн для предобработки данных:

In [ ]:
preprocessor = Pipeline(steps=[('', )])

### Напишем функцию для кросс-валидации:

In [ ]:
def objective_cv(trial, objective, n_splits):
    fold = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    scores = []
    
    for train_idx, valid_idx in fold.split(range(len(train_data))):
        train_features = train_data.iloc[train_idx].drop([''], axis=1)
        valid_features = train_data.iloc[valid_idx].drop([''], axis=1)
        train_target = train_data.iloc[train_idx]['']
        valid_target = train_data.iloc[valid_idx]['']

        dataset = ((train_features, train_target), (valid_features, valid_target))
        scores.append(objective(trial, dataset))
    return np.mean(scores)

### Создадим и обучим наивную модель:

При помощи перебора гиперпараметров найдём наилучшую наивную модель решений

In [ ]:
def objective(trial, dataset):
    (train_features, train_target), (valid_features, valid_target) = dataset
    
    regressor = DummyRegressor()
    regressor.strategy = trial.suggest_categorical('strategy', ['mean', 'median'])
        
    model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', regressor)])
    model.fit(train_features, train_target)
    return mean_absolute_error(valid_target, model.predict(valid_features))

study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(partial(objective_cv, objective=objective, n_splits=5), n_trials=2)

Добавим результаты кросс-валидации в таблицу

In [ ]:
model_params['DummyRegressor'] = study.best_trial.params
cv_results.loc['DummyRegressor', 'MAE'] = study.best_trial.values[0]
cv_results

### Создадим базовой нейронную сеть:

In [ ]:
# Класс полносвязной нейронной сети с произвольным числом слоёв и нейронов на каждом слое
class Net(nn.Module):
    def __init__(self, layers_neurons, dropout_layers, act_functions):
        super(Net, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(layers_neurons[i], layers_neurons[i+1]) for i in range(len(layers_neurons) - 1)])
        self.dropout_layers = nn.ModuleList(dropout_layers)
        self.act_functions = nn.ModuleList(act_functions)
        
    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            x = self.dropout_layers[i](x)
            x = self.act_functions[i](x)
            
        return x
    
# Класс модели нейронной сети для работы с библиотекой sklearn
class NNRegressor(BaseEstimator):
    def __init__(self, net, optimizer, num_epochs=1000, batch_size=512, verbose=False, verbose_epochs=100, **parameters):
        self.net = net
        self.optimizer = optimizer
        self.loss = nn.MSELoss()
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.verbose_epochs = verbose_epochs
        
        if 'dropout_probs' in parameters:
            for i in range(len(parameters['dropout_probs'])):
                self.net.dropout_layers[i].p = parameters['dropout_probs'][i]
        
    def get_params(self, deep=True):
        return {
            'net' : self.net,
            'optimizer': self.optimizer,
            'loss': self.loss,
            'num_epochs': self.num_epochs,
            'batch_size': self.batch_size,
            'verbose': self.verbose,
            'verbose_epochs': self.verbose_epochs
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
            
        if 'dropout_probs' in parameters:
            for i in range(len(parameters['dropout_probs'])):
                self.net.dropout_layers[i].p = parameters['dropout_probs'][i]
            
        return self
        
    def fit(self, X_train, y_train):                               
        if isinstance(X_train, (pd.DataFrame, pd.Series)):
            X_train = X_train.to_numpy(copy=True)
        if isinstance(y_train, (pd.DataFrame, pd.Series)):
            y_train = y_train.to_numpy(copy=True)

        X_train = torch.FloatTensor(X_train)
        y_train = torch.FloatTensor(y_train)
        
        num_batches = math.ceil(len(X_train) / self.batch_size)
        self.net.train()
        
        for epoch in range(self.num_epochs):
            order = np.random.permutation(len(X_train))
            
            for batch_idx in range(num_batches):
                start_index = batch_idx * self.batch_size
                self.optimizer.zero_grad()
        
                batch_indexes = order[start_index:start_index+self.batch_size]
                X_batch = X_train[batch_indexes]
                y_batch = y_train[batch_indexes]

                preds = self.net.forward(X_batch).flatten()
                loss_value = self.loss(preds, y_batch)
                loss_value.backward()
                self.optimizer.step()

            if self.verbose and epoch % self.verbose_epochs == 0:
                print(loss_value)

    def predict(self, X_test):
        if isinstance(X_test, (pd.DataFrame, pd.Series)):
            X_test = X_test.to_numpy(copy=True)

        self.net.eval()
        return self.net.forward(torch.FloatTensor(X_test)).detach().numpy().flatten()

In [ ]:
net = Net([5, 100, 100, 1], [nn.Dropout(0.5), nn.Dropout(0.5), nn.Identity()], [nn.ReLU(), nn.ReLU(), nn.Identity()])
nn.init.kaiming_uniform_(net.layers[0].weight, mode='fan_in', nonlinearity='relu')
nn.init.kaiming_uniform_(net.layers[1].weight, mode='fan_in', nonlinearity='relu') 
net

###  Проанализируем результаты моделирования и выберем наилучшую модель:

In [ ]:
results = study.trials_dataframe()
results.head()

### Краткий вывод:

## Тестирование и анализ наилучшей модели:

In [ ]:
test_results = pd.DataFrame(columns=['MAE', 'R2'])

### Обучим наилучшую модель на объединении обучающей и валидационной выборок:

In [ ]:
test_features = test_data.drop(['temperature_last'], axis=1)
test_target = test_data['temperature_last']

best_model = LGBMRegressor(**model_params['LGBMRegressor'])
best_model.fit(train_data.drop(['temperature_last'], axis=1), train_data['temperature_last'])
best_predictions = best_model.predict(test_features) 

test_results.loc['', 'MAE'] = mean_absolute_error(test_target, best_predictions)
test_results.loc['', 'R2'] = r2_score(test_target, best_predictions)
test_results

### Протестируем наилучшую модель на тестовой выборке:

In [ ]:
probabilities_one_valid = forest_model.predict_proba(valid_features)[:, 1]
fpr, tpr, thresholds = roc_curve(valid_target, probabilities_one_valid)

plt.figure()
# ROC-кривая модели RandomForestClassifier
plt.plot(fpr, tpr)

# ROC-кривая случайной модели (выглядит как прямая)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim(0, 1.0)
plt.ylim(0, 1.0)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

print('Площадь под ROC-кривой (AUC-ROC) равна:', roc_auc_score(valid_target, probabilities_one_valid))

### Проведём графический анализ:

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test,
                                                         y_pred,
                                                         normalize='all'),
                                                         display_labels=['at_fault', 'not at_fault']).plot();

In [ ]:
PrecisionRecallDisplay.from_estimator(pipeline, X_test, y_test);

### Проанализируем важность основных факторов, влияющих на... :

In [ ]:
ax = plot_importance(classifier, ignore_zero=False)
ax.set_yticklabels(pipeline[:-1].get_feature_names_out());

### Проанализируем зависимость целевого признака ` ` от ...:

In [ ]:
PartialDependenceDisplay.from_estimator(pipeline, test_features, ['vehicle_age'], line_kw={"color": "red"});

### Краткий вывод:

## Общий вывод: